In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from cvx_portfolio.costs import TcostModel
from cvx_portfolio.returns import MarketReturns
from cvx_portfolio.simulator import MarketSimulator
from cvx_portfolio.policies import Hold,PeriodicRebalance

plotdir = '../portfolio/plots/'
datadir= '../equity_data_tool/'

## Download data using ee103 code (to be integrated) 

In [2]:
sigmas=pd.read_csv(datadir+'sigmas.txt',index_col=0,parse_dates=[0])
returns=pd.read_csv(datadir+'returns.txt',index_col=0,parse_dates=[0])
volumes=pd.read_csv(datadir+'volumes.txt',index_col=0,parse_dates=[0])
prices=pd.read_csv(datadir+'prices.txt',index_col=0,parse_dates=[0])

## Market simulator 

In [3]:
a = pd.DataFrame(data=5e-4, index=returns.index, columns=returns.columns)
b = pd.DataFrame(data=1, index=returns.index, columns=returns.columns)
s = pd.DataFrame(data=0, index=returns.index, columns=returns.columns)

simulated_tcost = TcostModel(volumes, sigmas, a, b, cash_key='USDOLLAR')
market_returns = MarketReturns(returns)

simulator = MarketSimulator(market_returns, volumes, [simulated_tcost], cash_key='USDOLLAR')

## Portfolio

In [4]:
small_fund = 1E8
big_fund = 1E10

p = pd.Series(index=returns.columns, data=1)/len(returns.columns)
weights=p/sum(p)

p_small = small_fund*p
p_large = big_fund*p

## Policy

In [5]:
def select_starts_periods(period_property, timestamps):
    selected = [timestamps[0]]
    last_day = timestamps[0]
    for day in timestamps[1:]:
        if not (period_property(day) == period_property(last_day)):
            selected.append(day)
            last_day=day
    return selected

In [31]:
selectors=pd.Series(index=['D','W','M','Q','Y'],
                    data=[lambda t: t.day,lambda t: t.week,lambda t: t.month,lambda t: t.quarter,lambda t: t.year])
rebalancing_times = selectors.apply(lambda x: select_starts_periods(x, returns.index) )
policies = rebalancing_times.apply(lambda x: PeriodicRebalance(target=weights, rebalancing_times=x))
policies['H'] = Hold()

## Simulations

In [32]:
five_years=returns.index[returns.index>"2012-01-01"]

In [33]:
res_small_fund=policies.apply(lambda x: simulator.run_backtest(p_small, five_years, policy=x))

In [34]:
res_large_fund=policies.apply(lambda x: simulator.run_backtest(p_large, five_years, policy=x))

In [35]:
res=pd.DataFrame({'$100M':res_small_fund, '$10B': res_large_fund})

## Results 

In [36]:
pandl = lambda res: 100*(res.profit)/res.initial_val
tcosts = lambda res: 100*res.simulator_TcostModel.sum().sum()/res.initial_val
tmp=pd.Panel({'Profit':res.applymap(pandl), 'TCosts':res.applymap(tcosts)})
result_df = tmp.swapaxes(0,1).swapaxes(1,2).to_frame().T
result_df

major       $100M                  $10B          
minor      Profit    TCosts      Profit    TCosts
D      120.697145  1.121385  117.639067  3.449154
W      121.600349  0.595479  119.477541  2.211464
M      121.890627  0.342165  120.329678  1.543922
Q      122.821538  0.221629  121.637714  1.141452
Y      122.715806  0.115432  122.058559  0.700706
H      122.552537  0.000000  122.552537  0.000000

In [37]:
print(result_df.to_latex(float_format='%.2f%%'))

\begin{tabular}{lrrrr}
\toprule
major &   \$100M &        &    \$10B &        \\
minor &  Profit & TCosts &  Profit & TCosts \\
\midrule
D & 120.70\% &  1.12\% & 117.64\% &  3.45\% \\
W & 121.60\% &  0.60\% & 119.48\% &  2.21\% \\
M & 121.89\% &  0.34\% & 120.33\% &  1.54\% \\
Q & 122.82\% &  0.22\% & 121.64\% &  1.14\% \\
Y & 122.72\% &  0.12\% & 122.06\% &  0.70\% \\
H & 122.55\% &  0.00\% & 122.55\% &  0.00\% \\
\bottomrule
\end{tabular}



# Plots 

In [38]:
offsetx = .12
offsety = .13

for ticker, values in result_df.iterrows():
    for size, coord in values.unstack().iterrows():
        x,y=coord['TCosts'],coord['P&L']
        if size=='$100M':
            line1,=plt.plot(x,y, 'ko', markersize=11, fillstyle='none', label='100M fund')
            plt.text(x-offsetx,y-offsety, ticker)

        if size=='$10B':
            line2,=plt.plot(x,y, 'ro', markersize=13, fillstyle='none', label='10B fund')
            plt.text(x-offsetx,y-offsety,ticker,color='r' )

#plt.xlim([-1, 10])
#plt.ylim([13, 25])
plt.legend(loc='upper right',handles=[line1, line2])
plt.xlabel("Simulated TCosts (%)")
plt.ylabel("Simulated Profit (%)")
plt.title("Uniform portfolio rebalancing (2012-2016)")

plt.savefig(plotdir+'pandl_vs_tcost.png')

KeyError: 'P&L'

In [ ]:
for k,results in res.loc['D'].iteritems():
    plt.figure()
    (100*(results.v-results.initial_val)/results.initial_val).plot(label='P&L (%)')
    (100*(-results.simulator_TcostModel.sum(1).cumsum())/results.initial_val).plot(style='r',label='cum. TCosts (%)')
    plt.legend(loc='lower right')
    plt.title('%s uniform daily rebalance'%k)
    plt.savefig(plotdir+'%s_pandl_tcost_tseries.png'%k[1:])